In [1]:
!pip install safetensors

In [2]:
import torch
from torch import nn
from transformers import BartForConditionalGeneration, BartConfig
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention
from transformers.models.bart.modeling_bart import BartLearnedPositionalEmbedding
from typing import List, Optional, Tuple
from transformers import PreTrainedTokenizerFast
from safetensors.torch import load_file

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 모델 구조 로드

In [4]:
# Longformer Attention 레이어 정의
class LongformerAttention(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.embed_dim = config.d_model
        self.longformer_self_attn = LongformerSelfAttention(config, layer_id=layer_id)
        self.output_projection = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:

        if attention_mask is not None:
            attention_mask = attention_mask.squeeze(dim=1)
            attention_mask = attention_mask[:, 0]

        is_index_masked = attention_mask < 0 if attention_mask is not None else None
        is_index_global_attn = attention_mask > 0 if attention_mask is not None else None
        is_global_attn = is_index_global_attn.flatten().any().item() if attention_mask is not None else False

        outputs = self.longformer_self_attn(
            hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            is_index_masked=is_index_masked,
            is_index_global_attn=is_index_global_attn,
            is_global_attn=is_global_attn,
            output_attentions=output_attentions,
        )

        attention_output = self.output_projection(outputs[0])
        return (attention_output,) + outputs[1:] if len(outputs) == 2 else (attention_output, None, None)

In [5]:
# Longformer Attention 적용 KoBART 모델 정의
class KoBARTWithLongformer(BartForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)

        if config.attention_mode == 'n2':
            pass
        else:
            # 포지셔널 임베딩 확장 (encoder)
            self.model.encoder.embed_positions = BartLearnedPositionalEmbedding(
                config.max_encoder_position_embeddings,  # num_embeddings
                config.d_model  # embedding_dim
            )

            # 인코더에 Longformer Attention 적용
            for i, encoder_layer in enumerate(self.model.encoder.layers):
                encoder_layer.self_attn = LongformerAttention(config, layer_id=i)

In [6]:
# Longformer Attention 적용 KoBART 설정
class KoBARTConfig(BartConfig):
    def __init__(self, attention_window: List[int] = None,
                 attention_dilation: List[int] = None,
                 autoregressive: bool = False,
                 attention_mode: str = 'sliding_chunks',
                 gradient_checkpointing: bool = False,
                 attention_probs_dropout_prob: float = 0.1,
                 **kwargs):
        super().__init__(**kwargs)
        self.attention_window = attention_window
        self.attention_dilation = attention_dilation
        self.autoregressive = autoregressive
        self.attention_mode = attention_mode
        self.gradient_checkpointing = gradient_checkpointing
        self.attention_probs_dropout_prob = attention_probs_dropout_prob

        assert self.attention_mode in ['sliding_chunks', 'n2']

In [7]:
# 모델 초기화 함수
def initialize_kobart_with_longformer():
    attention_window_size = 512  # 짝수
    max_position_embeddings = 4104  # 1026의 배수

    # KoBART 기본 설정
    config = KoBARTConfig(
        vocab_size=30000,  # KoBART 기본 vocab 크기
        d_model=768,  # Hidden Dimension
        encoder_layers=6,  # 인코더 레이어 수
        decoder_layers=6,  # 디코더 레이어 수
        encoder_attention_heads=16,  # Attention Head 수
        decoder_attention_heads=16,  # Attention Head 수
        encoder_ffn_dim=3072,  # FFN 차원
        decoder_ffn_dim=3072,  # FFN 차원
        attention_window=[attention_window_size] * 6,  # Longformer Attention Window 설정
        max_encoder_position_embeddings=max_position_embeddings,  # 확장된 포지셔널 임베딩
        max_decoder_position_embeddings=1026,  # 디코더 크기 유지
        attention_mode='sliding_chunks',
        gradient_checkpointing=True,
        attention_probs_dropout_prob=0.1  # 드롭아웃 설정
    )

    # Tokenizer 및 모델 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
    model = KoBARTWithLongformer.from_pretrained(
        "gogamza/kobart-base-v2",
        config=config,
        ignore_mismatched_sizes=True  # 초기 로드 시 불일치 무시
    )

    # 인코더 포지셔널 임베딩 확장
    current_max_pos, embed_size = model.model.encoder.embed_positions.weight.shape
    print(f"Current encoder max pos: {current_max_pos}")
    new_encoder_pos_embed = model.model.encoder.embed_positions.weight.new_empty(
        config.max_encoder_position_embeddings + 2, embed_size
    )
    # 기존 가중치 복사
    new_encoder_pos_embed[:current_max_pos] = model.model.encoder.embed_positions.weight
    # 확장된 부분 랜덤 초기화
    nn.init.normal_(new_encoder_pos_embed[current_max_pos:], mean=0.0, std=0.02)
    model.model.encoder.embed_positions.weight.data = new_encoder_pos_embed

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    return model, tokenizer

#### 저장된 가중치 로드

In [8]:
# 모델 가중치 로드 함수
def load_model_weights(model, model_path):
    # 모델에 가중치 로드
    state_dict = load_file(f"{model_path}/model.safetensors")
    model.load_state_dict(state_dict, strict=False)

    return model

# 저장된 모델 경로
model_path = "/content/drive/MyDrive/응용자연어처리/project/Longformer_KoBART_finetuning_v6"

# 저장된 가중치와 토크나이저 로드
model, tokenizer = initialize_kobart_with_longformer()

model = load_model_weights(model, model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

Some weights of KoBARTWithLongformer were not initialized from the model checkpoint at gogamza/kobart-base-v2 and are newly initialized: ['encoder.layers.0.self_attn.longformer_self_attn.key.bias', 'encoder.layers.0.self_attn.longformer_self_attn.key.weight', 'encoder.layers.0.self_attn.longformer_self_attn.key_global.bias', 'encoder.layers.0.self_attn.longformer_self_attn.key_global.weight', 'encoder.layers.0.self_attn.longformer_self_attn.query.bias', 'encoder.layers.0.self_attn.longformer_self_attn.query.weight', 'encoder.layers.0.self_attn.longformer_self_attn.query_global.bias', 'encoder.layers.0.self_attn.longformer_self_attn.query_global.weight', 'encoder.layers.0.self_attn.longformer_self_attn.value.bias', 'encoder.layers.0.self_attn.longformer_self_attn.value.weight', 'encoder.layers.0.self_attn.longformer_self_attn.value_global.bias', 'encoder.layers.0.self_attn.longformer_self_attn.value_global.weight', 'encoder.layers.0.self_attn.output_projection.bias', 'encoder.layers.0.s

Current encoder max pos: 4106


#### 요약문 생성

In [9]:
'''
# 요약문 생성 함수
def generate_summary(model, tokenizer, text):
    device = next(model.parameters()).device
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding="max_length"
    ).to(device)
    inputs.setdefault("attention_mask", (inputs["input_ids"] != tokenizer.pad_token_id).long())

    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
'''

'\n# 요약문 생성 함수\ndef generate_summary(model, tokenizer, text):\n    device = next(model.parameters()).device\n    inputs = tokenizer(\n        text,\n        return_tensors="pt",\n        max_length=1024,\n        truncation=True,\n        padding="max_length"\n    ).to(device)\n    inputs.setdefault("attention_mask", (inputs["input_ids"] != tokenizer.pad_token_id).long())\n\n    summary_ids = model.generate(\n        inputs["input_ids"],\n        attention_mask=inputs["attention_mask"],\n        max_length=128,\n        num_beams=4,\n        early_stopping=True\n    )\n    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)\n'

In [10]:
from torch.utils.data import DataLoader
import pandas as pd

In [11]:
# 테스트 데이터셋 로드
df = pd.read_csv("/content/drive/MyDrive/응용자연어처리/project/data/medium_test.csv")

In [12]:
# 배치 요약문 생성 함수
def generate_batch_summaries(model, tokenizer, texts, device, batch_size=16):
    dataloader = DataLoader(texts, batch_size=batch_size)
    summaries = []
    for batch in dataloader:
        # 텍스트를 배치로 토큰화
        inputs = tokenizer(
            list(batch),
            return_tensors="pt",
            max_length=1024,
            truncation=True,
            padding="max_length"
        ).to(device)

        # 모델로 요약 생성
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )

        # 요약문 디코딩
        batch_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        summaries.extend(batch_summaries)
    return summaries

# GPU/CPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 배치 처리 및 요약문 생성
batch_size = 16
df["generated_summary"] = generate_batch_summaries(
    model, tokenizer, df["contents"].tolist(), device, batch_size
)

In [13]:
df.head()

,contents,summary,char_length,length,generated_summary
0,황인옥 세계여행 선언한 주부 이야기 대한민국연극제 대구 대표 참가 최우수연기상 ‘툇...,지난 31일에 열린 시상식에서 극단 온누리가 12년 만의 출전으로 제36회 대구연극...,1017,medium,제36회 대구연극제에서 대상을 수상한 극단 온누리의 연극외출연출 '제3회 대구연극제...
1,임재업 신ㆎ 구 이사장간 '고소전' 치달아 (동양일보 임재업 기자) 청주첨단산업용품...,청주 첨단산업용품 상사 협동조합이 청주 테크노폴리스 지역에 조성 예정인 산업용품단지...,1069,medium,청주첨단산업용품상사협동조합(이하 조합)이 청주테크노폴리스지역에 조성 예정인 산업용품...
2,도-경기관광公 ‘경기그랜드투어-해설이 있는 여행’ 출시 ▲ 경기관광공사가 명사와 함...,"경기도와 경기관광공사는 남한산성 성곽길을 걸으며 역사 이야기를 듣고, 화담숲에서 나...",1076,medium,경기관광공사가 명사와 함께하는 역사문화유적 투어 상품 '경기그랜드투어-해설이 있는 ...
3,[서울경제] 최근 경기 악화로 지난해 경북 김천·구미 지역에서 노동청에 임금 체불 ...,"구미 김천지역 체불임금이 5년전보다 2개 가까이 증가한 것으로 나타나, 구미고용노동...",1005,medium,"경북 김천·구미 지역에서 임금체불 신고를 한 근로자는 3,482명으로 5년 전보다 ..."
4,19일 여당인 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 ...,"19일 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 대해 ""...",1439,medium,"더불어민주당은 19일 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 대해 ""..."


In [14]:
summaries = df["summary"]
generated_summaries = df["generated_summary"]

#### 성능평가 - medium

ROUGE: https://github.com/HeegyuKim/korouge

In [15]:
!pip install korouge_score

In [16]:
import numpy as np
from korouge_score import rouge_scorer

In [17]:
# 한국어 ROUGE
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL", "rougeLsum"])

In [18]:
# ROUGE 점수 계산
rouge_scores = []
for reference, generated in zip(df["summary"], df["generated_summary"]):
    scores = scorer.score(reference, generated)
    rouge_scores.append(scores)

# 결과 확인
for i, scores in enumerate(rouge_scores[:5]):
    print(f"Sample {i + 1}:")
    print(scores)
    print()

Sample 1:
{'rouge1': Score(precision=0.375, recall=0.3, fmeasure=0.33333333333333326), 'rouge2': Score(precision=0.13333333333333333, recall=0.10526315789473684, fmeasure=0.11764705882352941), 'rougeL': Score(precision=0.3125, recall=0.25, fmeasure=0.2777777777777778), 'rougeLsum': Score(precision=0.3125, recall=0.25, fmeasure=0.2777777777777778)}

Sample 2:
{'rouge1': Score(precision=0.36363636363636365, recall=0.38095238095238093, fmeasure=0.37209302325581395), 'rouge2': Score(precision=0.23809523809523808, recall=0.25, fmeasure=0.24390243902439024), 'rougeL': Score(precision=0.2727272727272727, recall=0.2857142857142857, fmeasure=0.2790697674418604), 'rougeLsum': Score(precision=0.2727272727272727, recall=0.2857142857142857, fmeasure=0.2790697674418604)}

Sample 3:
{'rouge1': Score(precision=0.2857142857142857, recall=0.2, fmeasure=0.23529411764705882), 'rouge2': Score(precision=0.1, recall=0.06896551724137931, fmeasure=0.08163265306122448), 'rougeL': Score(precision=0.2380952380952

In [19]:
# ROUGE 점수 평균 계산 (fmeasure 평균)
metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
average_scores = {metric: 0 for metric in metrics}

# 총 샘플 수
n = len(rouge_scores)

# 각 메트릭의 fmeasure 합산
for score in rouge_scores:
    for metric in metrics:
        average_scores[metric] += getattr(score[metric], "fmeasure", 0)

# 각 메트릭의 평균 계산
for metric in metrics:
    average_scores[metric] /= n

print(average_scores)

{'rouge1': 0.25895039622320637, 'rouge2': 0.13557107896900392, 'rougeL': 0.2308738354484085, 'rougeLsum': 0.2309329271208124}


BERT-Score: https://github.com/Tiiiger/bert_score

In [20]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [21]:
# BERT-Score
bert_tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
bert_model = BertModel.from_pretrained("beomi/kcbert-base").eval()

def get_bert_embeddings(sentence, tokenizer, model):
    # 문장을 토큰화하고 BERT 입력 형식으로 변환
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

    # BERT 모델을 통해 문장의 임베딩 얻기
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0)

    return embeddings.numpy()

def calculate_bert_score(sentence1, sentence2, tokenizer, model):
    # 각 문장의 BERT 임베딩 벡터 얻기
    embedding1 = get_bert_embeddings(sentence1, tokenizer, model)
    embedding2 = get_bert_embeddings(sentence2, tokenizer, model)

    # 코사인 유사도 계산 (1 - cosine distance)
    score = 1 - cosine(embedding1, embedding2)

    return score

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [22]:
# BERT-Score 점수 계산
df["bert_score"] = df.apply(lambda row: calculate_bert_score(row["summary"], row["generated_summary"], bert_tokenizer, bert_model), axis=1)
print(df.head())

                                            contents  \
0  황인옥 세계여행 선언한 주부 이야기 대한민국연극제 대구 대표 참가 최우수연기상 ‘툇...   
1  임재업 신ㆎ 구 이사장간 '고소전' 치달아 (동양일보 임재업 기자) 청주첨단산업용품...   
2  도-경기관광公 ‘경기그랜드투어-해설이 있는 여행’ 출시 ▲ 경기관광공사가 명사와 함...   
3  [서울경제] 최근 경기 악화로 지난해 경북 김천·구미 지역에서 노동청에 임금 체불 ...   
4  19일 여당인 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 ...   

                                             summary  char_length  length  \
0  지난 31일에 열린 시상식에서 극단 온누리가 12년 만의 출전으로 제36회 대구연극...         1017  medium   
1  청주 첨단산업용품 상사 협동조합이 청주 테크노폴리스 지역에 조성 예정인 산업용품단지...         1069  medium   
2  경기도와 경기관광공사는 남한산성 성곽길을 걸으며 역사 이야기를 듣고, 화담숲에서 나...         1076  medium   
3  구미 김천지역 체불임금이 5년전보다 2개 가까이 증가한 것으로 나타나, 구미고용노동...         1005  medium   
4  19일 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 대해 "...         1439  medium   

                                   generated_summary  bert_score  
0  제36회 대구연극제에서 대상을 수상한 극단 온누리의 연극외출연출 '제3회 대구연극제...    0.877616  
1  청주첨단산업용품상사협동조합(이하 조합)이 청주테크노폴리스지역에 조성 예정인 산업용품...    0.917411  

In [23]:
# BERT-Score 평균
df["bert_score"].mean()

0.8598117142139095